<center>
<img src="./pict/folium_logo.png">
<br />
<br />

In [12]:
# !pip install folium
# установка Folium, или так: !conda install folium -c conda-forge

In [13]:
# arcgis, pysal, geopandas 

In [1]:
import folium
import geopy
# объединенное API для популярных гео-кодирующих сервисов
from geopy.geocoders import Yandex, GoogleV3

# https://developers.google.com/maps/documentation/geocoding/start - получить свой API Key
# geolocator_google = GoogleV3(api_key='')

In [2]:
# !pip install xlrd==1.0.0

In [2]:
# https://mash.ru/letter/coronavirus-2/ - данные взяты отсюда

import pandas as pd

coronavirus_data = pd.read_excel('./data/coronavirus_cases.xlsx')
coronavirus_data.head()

<ipython-input-2-e8055e58edb3>:5: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  coronavirus_data = pd.read_excel('./data/coronavirus_cases.xlsx')


,#,address,date
0,1,"3-й квартал Капотни, 3",28-30 марта
1,2,"4-й квартал Капотни, 9",28-30 марта
2,3,"Авиаторов,6, к1",28-30 марта
3,4,"Адмирала Лазарева, 38",28-30 марта
4,5,"Академика Комарова, 7а",28-30 марта


In [3]:
geolocator_google.geocode('Москва, Китай-город')

NameError: name 'geolocator_google' is not defined

In [4]:
geolocator_google.geocode('Московский физико-технический институт')

NameError: name 'geolocator_google' is not defined

In [5]:
from tqdm import tqdm_notebook

lat = []
long = []
strange_address = []

for i in tqdm_notebook(range(coronavirus_data.shape[0])):
    
    address = 'Москва, ' + coronavirus_data.address.iloc[i]
    try:
        location = geolocator_google.geocode(address) # геокодирование
        lat.append(location.latitude)
        long.append(location.longitude)
        
    except:
        strange_address.append(address)
        lat.append('NaN')
        long.append('NaN')
        
coronavirus_data['lat'] = lat
coronavirus_data['long'] = long

ModuleNotFoundError: No module named 'tqdm'

In [6]:
coronavirus_data = pd.read_csv('./data/coronavirus_cases.csv')

In [7]:
coronavirus_data.to_csv('./data/coronavirus_cases.csv', index=False)

In [4]:
from folium.features import DivIcon

m = folium.Map(location=[55.755826, 37.6172999], # Москва
               zoom_start=10)

folium.Marker([55.9297243, 37.5199434], 
              tooltip='МФТИ',
              icon=folium.Icon(color='green', icon=None)).add_to(m)

folium.Marker([55.752023, 37.6174994], 
              tooltip='Кремль',
              icon=folium.Icon(color='red', icon='star')).add_to(m)

folium.CircleMarker(location = [55.9297243, 37.5199434], 
                    radius = 50).add_to(m)

folium.PolyLine(locations = [(55.929724, 37.5199434), (55.752023, 37.6174994)], 
                line_opacity = 0.5).add_to(m)

folium.map.Marker(
    [55.755826, 37.6172999],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:red">Какой-нибудь текст </div>',
        )
    ).add_to(m)


m

In [9]:
coronavirus_data.date.unique()

array(['28-30 марта', '25 марта-27 марта', '24 марта'], dtype=object)

In [10]:
colors = dict(zip(coronavirus_data.date.unique(), ['green', 'darkred', 'blue']))

In [11]:
coronavirus_data = coronavirus_data[(coronavirus_data.lat != 'NaN') & (coronavirus_data.long != 'NaN')]

In [12]:
coronavirus_data = coronavirus_data.fillna(1)

In [13]:
coronavirus_data

,#,address,date,lat,long
0,1,"3-й квартал Капотни, 3",28-30 марта,55.635714,37.802126
1,2,"4-й квартал Капотни, 9",28-30 марта,55.636395,37.792573
2,3,"Авиаторов,6, к1",28-30 марта,55.647897,37.394463
3,4,"Адмирала Лазарева, 38",28-30 марта,55.545078,37.524365
4,5,"Академика Комарова, 7а",28-30 марта,55.828914,37.592907
...,...,...,...,...,...
375,376,"Шереметьевская, 27",24 марта,55.803321,37.614746
376,377,"Шмитовский проезд, 16",24 марта,55.757758,37.546720
377,378,"Шоссейная, 50",24 марта,55.679432,37.721865
378,379,"Щелковское шоссе, 98/57",24 марта,55.811789,37.827629


In [14]:
m = folium.Map(location=[55.755826, 37.6172999], # Москва
               zoom_start=10)

folium.Marker([55.9297243, 37.5199434], 
              tooltip='МФТИ',
              icon=folium.Icon(color='green', icon=None)).add_to(m)

folium.Marker([55.752023, 37.6174994], 
              tooltip='Кремль',
              icon=folium.Icon(color='red', icon='star')).add_to(m)

for i, row in coronavirus_data.iterrows():
    
    color = colors[row.date]
    folium.Marker([row.lat, row.long], 
                  tooltip=row.date,
                  icon=folium.Icon(color=color, icon='None'),
                  popup=folium.Popup('<b> Адрес: </b>' + '<br>' +
                                     row.address, 
                                     max_width=500)).add_to(m)

m

In [15]:
m.save('./data/coronavirus_cases_map.html') # конвертирование в html

In [16]:
from folium.plugins import HeatMap 

coronavirus_heatmap = coronavirus_data[['lat', 'long']].copy()
coronavirus_heatmap['weight'] = 1
coronavirus_heatmap = coronavirus_heatmap.values.tolist()

m = folium.Map(location=[55.755826, 37.6172999], tiles='stamentoner',
               zoom_start=10)

HeatMap(coronavirus_heatmap).add_to(m)

m

In [17]:
from folium.plugins import HeatMapWithTime

coronavirus_heatmap = coronavirus_data[['date', 'lat', 'long']].copy()
coronavirus_heatmap['weight'] = 1

coronavirus_heatmaptime = []
for dates in [['24 марта'], 
              ['24 марта', '25 марта-27 марта'], 
              ['24 марта', '25 марта-27 марта', '28-30 марта']]:
    coronavirus_heatmaptime.append(coronavirus_heatmap[coronavirus_heatmap.date.isin(dates)][['lat', 
                                                                         'long', 'weight']].values.tolist())
    
m = folium.Map(location=[55.755826, 37.6172999], 
               zoom_start=9)

HeatMapWithTime(coronavirus_heatmaptime).add_to(m)

m

In [18]:
# Как найти полигоны?

# Заходим сюда: https://nominatim.openstreetmap.org/search.php?q=&polygon_geojson=1&viewbox=
# Ищем необходимый объект
# В описании объекта находим код OSM 
# Заходим сюда: http://polygons.openstreetmap.fr/index.py 
# Вбиваем код OSM
# Получаем GeoJSON 
# Profit! 

In [5]:
import json

with open('./data/moscow.json') as json_file:
    moscow_polygon = json.load(json_file)
    
with open('./data/new_moscow.json') as json_file:
    new_moscow_polygon = json.load(json_file)
    
with open('./data/tver_region.json') as json_file:
    tver_region_polygon = json.load(json_file)
    
m = folium.Map(location=[55.755826, 37.6172999], # Москва
               zoom_start=7)

m.choropleth(new_moscow_polygon, fill_color='yellow')
m.choropleth(moscow_polygon, fill_color='brown')
m.choropleth(tver_region_polygon, fill_color='blue')

folium.map.Marker(
    [55.755826, 37.6172999],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:green"> Москва </div>',
        )
    ).add_to(m)

folium.map.Marker(
    [55.4903205, 37.3032108], # Троицк
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:red"> Троицк </div>',
        )
    ).add_to(m)


folium.map.Marker(
    [56.85872140000001, 35.9175965], # Тверь
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt; color:white"> Тверь </div>',
        )
    ).add_to(m)

m